## Comparación: Sesgo Simple vs Semántico

Este notebook visualiza los resultados del experimento comparativo.

### 1: Setup

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats

In [ ]:
# Cargar datos
df = pd.read_csv("../data/results/semantic_comparison/comparison_results.csv")

print(f" Datos cargados: {len(df)} generaciones")
print(f"   Condiciones: {df['condition'].unique()}")
print(f"   Prompts únicos: {df['prompt'].nunique()}")

### Celda 2: Estadísticas Descriptivas

In [ ]:
summary = df.groupby('condition').agg({
    'distinct_2': ['mean', 'std', 'count'],
    'sentiment_polarity': ['mean', 'std'],
    'repetition_rate': ['mean', 'std'],
    'length': ['mean', 'std']
}).round(3)

print("Estadísticas por condición:")
print(summary)

### Celda 3: Visualización Comparativa

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Comparación: Sesgo Simple vs Sesgo Semántico', 
             fontsize=16, fontweight='bold')

# Plot 1: Diversidad Léxica
sns.boxplot(data=df, x='condition', y='distinct_2', ax=axes[0, 0])
axes[0, 0].set_title('Diversidad Léxica (Distinct-2)')
axes[0, 0].set_xlabel('Condición')
axes[0, 0].set_ylabel('Distinct-2')

# Plot 2: Polaridad de Sentimiento
sns.boxplot(data=df, x='condition', y='sentiment_polarity', ax=axes[0, 1])
axes[0, 1].set_title('Polaridad de Sentimiento')
axes[0, 1].set_xlabel('Condición')
axes[0, 1].set_ylabel('Polaridad')

# Plot 3: Tasa de Repetición
sns.boxplot(data=df, x='condition', y='repetition_rate', ax=axes[1, 0])
axes[1, 0].set_title('Tasa de Repetición')
axes[1, 0].set_xlabel('Condición')
axes[1, 0].set_ylabel('Repetition Rate')

# Plot 4: Longitud
sns.boxplot(data=df, x='condition', y='length', ax=axes[1, 1])
axes[1, 1].set_title('Longitud del Texto')
axes[1, 1].set_xlabel('Condición')
axes[1, 1].set_ylabel('Número de Tokens')

plt.tight_layout()
plt.savefig('../data/results/semantic_comparison/comparison_boxplots.png', 
            dpi=300, bbox_inches='tight')
plt.show()


### Celda 4: Tests Estadísticos

In [ ]:
print("Tests Estadísticos (t-test independiente):")
print("="*60)

simple = df[df['condition'] == 'simple_bias']
semantic = df[df['condition'] == 'semantic_bias']

for metric in ['distinct_2', 'sentiment_polarity', 'repetition_rate', 'length']:
    simple_vals = simple[metric].dropna()
    semantic_vals = semantic[metric].dropna()
    
    t_stat, p_value = stats.ttest_ind(simple_vals, semantic_vals)
    
    # Effect size (Cohen's d)
    pooled_std = np.sqrt((simple_vals.std()**2 + semantic_vals.std()**2) / 2)
    cohens_d = (semantic_vals.mean() - simple_vals.mean()) / pooled_std
    
    print(f"\n{metric}:")
    print(f"  Simple:   M={simple_vals.mean():.4f}, SD={simple_vals.std():.4f}")
    print(f"  Semantic: M={semantic_vals.mean():.4f}, SD={semantic_vals.std():.4f}")
    print(f"  t({len(simple_vals) + len(semantic_vals) - 2})={t_stat:.3f}, p={p_value:.4f}")
    print(f"  Cohen's d={cohens_d:.3f}")
    
    if p_value < 0.001:
        print(f"  Resultado: Diferencia MUY significativa ***")
    elif p_value < 0.01:
        print(f"  Resultado: Diferencia significativa **")
    elif p_value < 0.05:
        print(f"  Resultado: Diferencia marginalmente significativa *")
    else:
        print(f"  Resultado: No significativo")
        

### Celda 5: Activación Semántica

In [ ]:
if 'semantic_activation_empathy' in df.columns:
    semantic_only = df[df['condition'] == 'semantic_bias'].copy()
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Histogram de activación
    axes[0].hist(semantic_only['semantic_activation_empathy'], bins=20, 
                 alpha=0.7, color='skyblue', edgecolor='black')
    axes[0].axvline(semantic_only['semantic_activation_empathy'].mean(), 
                    color='red', linestyle='--', linewidth=2, 
                    label=f"Media: {semantic_only['semantic_activation_empathy'].mean():.3f}")
    axes[0].set_xlabel('Activación de Empatía')
    axes[0].set_ylabel('Frecuencia')
    axes[0].set_title('Distribución de Activación Semántica (Empatía)')
    axes[0].legend()
    axes[0].grid(alpha=0.3)
    
    # Categoría dominante
    category_counts = semantic_only['dominant_semantic_category'].value_counts()
    axes[1].bar(category_counts.index, category_counts.values, 
                alpha=0.7, color='lightcoral', edgecolor='black')
    axes[1].set_xlabel('Categoría Semántica')
    axes[1].set_ylabel('Frecuencia')
    axes[1].set_title('Categoría Semántica Dominante')
    axes[1].tick_params(axis='x', rotation=45)
    axes[1].grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('../data/results/semantic_comparison/semantic_activation.png',
                dpi=300, bbox_inches='tight')
    plt.show()
    

### Celda 6: Análisis por Categoría de Prompt

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

# Agrupar por categoría de prompt y condición
category_comparison = df.groupby(['prompt_category', 'condition'])['distinct_2'].mean().unstack()

category_comparison.plot(kind='bar', ax=ax, alpha=0.8)
ax.set_title('Diversidad Léxica por Categoría de Prompt', fontsize=14, fontweight='bold')
ax.set_xlabel('Categoría de Prompt')
ax.set_ylabel('Distinct-2 (promedio)')
ax.legend(title='Condición', labels=['Simple', 'Semántico'])
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('../data/results/semantic_comparison/by_prompt_category.png',
            dpi=300, bbox_inches='tight')
plt.show()

print("\n Análisis completo")
